# Computer Vision Recap - Segmentation

## Lecture 13: Isotropic Nonlinear Diffusion

### Isotropic Nonlinear Diffusion


Avoid drawbacks of linear diffusion: delocalisation and blurring of edges
$$ \partial_t u = div( g(|\nabla u|^2) \; \nabla u ) $$

Diffusivity $g(|\nabla u|^2) := \frac{1}{1+|\nabla u|^2/\lambda^2}$ decreasing in $|\nabla u|^2$ 


<img src="./images/isodiff.png" width=600>
<br>

- Forward diffusion if $\Phi'(u_x) > 0$ for $|\nabla u| < \lambda$
- Backward diffusion if $\Phi'(u_x) < 0$ for $|\nabla u| > \lambda$

### Regularised Nonlinear Diffusion

- Reduce staircasing artifacts and problems with noise

$$ \partial_t u = div( g(|\nabla u_\sigma|^2) \; \nabla u ) $$ with $$ u_\sigma = K_\sigma*u $$

<br>

Theoretical Results

- Stability under perturbations of initial image
- Preservation of average grey value
- Maximum-minimum principle
- Stable
- Image converges to average grey level

<br><br>
## Lecture 14: Anisotropic Nonlinear Diffusion

**Motivation**
- Isotropic nonlinear diff. have difficulties with noisy images and cannot enhance coherent flow-like structures
- We want to smooth only along edge directions but no across. 

<br>

### Edge-enhancing Anisotropic Diffusion (EED)

**Goal**
- Smooth prefently within regions
- Inhibit diffusion perpendicular to edges

$$ D(\nabla u_\sigma) = (v_1 \;\; v_2) \begin{bmatrix} \lambda_1&0\\0&\lambda_2 \end{bmatrix} (v_1 \;\; v_2)^\top = \lambda_1 v_1 v_1^\top + \lambda_2 v_2 v_2^\top $$

- $\lambda_1 = 1$ (diffusion along edge), $v_1 \parallel \nabla u_\sigma$ 
- $\lambda_2 = g(|\nabla u|^2)$ (diffusion across edge), $v_2 \perp \nabla u_\sigma$

<img src="./images/EED.png" width=300>

<br>

**Problems**:
- Large convolution kernels cancel adjacent gradients with the same direction but opposite orientation
- EED is not good for closing interrupted lines

**Analysing Local Structure**
- small $\sigma$: sensitive to noise
- big $\sigma$: cancellation of adjacent gradients with opposite directions

<br>

### Coherence-Enhancing Anisotropic Diffusion (CED)

basically a 1-D smoothing along flow-like structures

- smooth mainly along coherent structures
- increase smoothing with coherence $(\mu_1-\mu_2)^2$ 

**Choice of Diffusion Tensor $D(J_\rho)$**

Coherence (anisotropy) $c=(\mu_1-\mu_2)^2$ are useful to analyse the gradient covariance (how allongated is the covariance?)

- eigenvectors from $J_\rho(\nabla u_\sigma) = K_\rho * (\nabla u_\sigma \nabla u_\sigma^\top)$
- Across coherent structures: $\lambda_1 := \alpha (small \; \alpha >0)$ 
- Along coherent structures: $\lambda_2 := \alpha + (1-\alpha) \exp( \frac{-C}{(\mu_1-\mu_2)^2} )$ 


<br><br>
## Lecture 15: Mumford-Shah Model (Segmentation)



### Mumford-Shah Model

Seeks segmentation (u,K) of some initial image $f:\Omega\rightarrow \mathbb R$ as the minimizer of

$$ E_{MS}(u,K) = \int_\Omega (u-f)^2 dx dy + \alpha \int_{\Omega\backslash K} |\nabla u|^2 dx dy + \lambda l(K)$$

- matematically extremely challenging

<br>

### Mumford-Shah Cartoon Model

Approximation for $\alpha\rightarrow \infty$:
$$ E_{MS}(u,K) = \int_\Omega (u-f)^2 dx dy + \lambda l(K)$$

which is sufficient to control the homogeneity within each segment and also minimize the length of the boundaries

#### Approximation method for minimization

A segmentation is called **2-normal** if every segmentation $K'$, that results from merging two neighbouring segments of $K$ has a large energy:
$$E(K') > E(K)$$

**Idea:** approximate global minimiser of cartoon model by local minimiser resulting from the restriction to 2-normal segmentations. Create 2-normal segmentations by region merging

Merging two regions yields:
$$ E(K\backslash \partial(\Omega_i,\Omega_j)) = E(K) + \frac{|\Omega_i|\cdot |\Omega_j|}{|\Omega_i|+|\Omega_j|} (u_i-u_j)^2  - \lambda l(\partial (\Omega_i,\Omega_j))$$

Solving $E(K\backslash \partial(\Omega_i,\Omega_j)) - E(K) = 0$ yields:
$$ \lambda_{i,j} = \frac{ \frac{|\Omega_i|\cdot |\Omega_j|}{|\Omega_i|+|\Omega_j|} (u_i-u_j)^2 }{l(\partial (\Omega_i,\Omega_j))} $$

<br>

### The Ambrosio-Tortorelli Mode



Approximation of the Mumford-Shah functional by:
$$ E_{MS-AT}(u,v) := \int_{\Omega} \left(  (u-f)^2 + \alpha\cdot v^2\cdot |\nabla u|^2 + \beta\cdot \left(c|\nabla v|^2+\dfrac{(1-v)^2}{4c}\right)  \right) dx dy$$

where v(x,y) is an edge indicator function ($v\rightarrow 0$ at edges and 1 within region)

- **Advantage**: smooth unkown function $v$ is mathm. more convenient than an unkown edge set $K$ 

<br>

Solving with Euler-Lagrange equations yields:
$$ 0 = \frac{1}{\alpha} (u-f) - div(v^2 \nabla u) $$
$$ 0 = \frac{\alpha}{\beta} v |\nabla u|^2 - \frac{1-v}{4c} - c\Delta v $$ 

with boundary conditions:
$$ 0 = n \cdot \nabla u \quad \quad 0=n\cdot\nabla v  $$

<br>

- The MS-AT energy is not jointly convex in $(u,v)$ and therefore may have multiple minima
- Consequently, the diffusion-reaction system may get stuck in a local minima

<br>

The Ambrosio-Tortorelli are related to the Perona-Malik (nonlinear isotropic diffusion) filter